第四回レポート課題
参考文献1を参考に誤差逆伝搬法について学習した。

1.計算グラフについての理解\
ニューラルネットワークにおける処理の一つ一つをレイヤという単位で考えることができる。これにより開発が容易になる。
逆伝搬により入力した値による出力値の偏微分の値を効率よく計算できる

2.乗算レイヤの実装

In [1]:
#乗算レイヤ
class MulLayer:
    def __init__(self):
        #入力値の保存用
        self.x = None 
        self.y = None
    
    #順伝播
    def forward(self, x, y):
        #入力値の保存
        self.x = x
        self.y = y
        #乗算
        out = x * y
        return out
    
    #逆伝播
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy

In [3]:
#実行
apple = 100
apple_num = 10
tax_rate = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

#forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax_rate)

print(price)

#backward
dprice = 1
dapple_price, dtax_rate = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax_rate)

1100.0
11.0 110.00000000000001 1000


確かに計算グラフ通りの値を出力している

3.加算レイヤの実装

In [28]:
class AddLayer():
    def __init__(self):
        pass
    
    def forward(self, x, y):
        #加算
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy    

In [29]:
#実行
apple = 100
apple_num = 10
orange = 150
orange_num = 25
tax_rate = 1.1

mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

#forward
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax_rate)

print(price)

#backward
dprice = 1
dall_price, dtax_rate = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dorange, dorange_num, dtax_rate)


5225.0
11.0 110.00000000000001 27.500000000000004 165.0 4750


このように出力値に対する各変数の偏微分した値をbackward()メソッドで求めることができる
また、forwardとbackwardで処理する順番が真逆であることも確認できた。

活性化関数の実装
非線形な関数についても考える。また、スカラーではなくNumpy配列を扱うことを考える。

4.RELUレイヤの実装\
$$y = x (x >  0)$$
$$y = 0 (x <= 0)$$
なので
$$\frac{\partial y}{\partial x} = 1 (x >  0)$$
$$\frac{\partial y}{\partial x} = 0 (x <= 0)$$
になることに注意\
ReLUレイヤはスイッチのような役割を果たす

In [30]:
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0) #0以下の場所をTrueとして記録した配列
        out = x.copy() #複製(入力値への影響をなくす(そのまま代入だと参照渡しになるのでcopy()を用いる))
        out[self.mask] = 0 #mask内でTrueの場所に0を代入
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0 #入力値が0以下は0に,0より大きい場合はそのまま
        dx = dout
        return dx

In [31]:
#実行
import numpy as np
x = np.array( [[ 1.0, -0.5], 
               [-2.0,  3.0]])

print(x)

relu_layer = Relu()

#順伝播
y = relu_layer.forward(x)
print(y)

print(relu_layer.mask)

#逆伝播
dout = np.array([[ 1.0, 1.0], 
               [1.0, 1.0]])
dx = relu_layer.backward(dout)
print(dx)

[[ 1.  -0.5]
 [-2.   3. ]]
[[1. 0.]
 [0. 3.]]
[[False  True]
 [ True False]]
[[1. 0.]
 [0. 1.]]


5.Sigmoidレイヤの実装\
Sigmoid関数は次のように表すことができる\
$$y = \frac{1}{1 + exp(-x)}$$
また逆伝播の出力は以下のようになる。導出は参考文献1を参考にした。\
$$\frac{\partial L}{\partial y} y^{2}exp(-x) = \frac{\partial L}{\partial y} y (1 - y)$$

In [32]:
class Sigmoid:
    def __init__(self):
        #出力結果(y)の保存用
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * self.out * (1.0 - self.out)
        return dx

参考文献
1. ゼロから作る Deep Learning Pythonで学ぶディープラーニングの理論と実装
2. https://qiita.com/mytk0u0/items/231807f4136b2b1681b0 (copyメソッドについて調べるのに利用)